In [ ]:
%env CUDA_VISIBLE_DEVICES=4

In [2]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd.profiler as profiler

import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms


LOG_PATH = "./data/profile_info/"
DATA_DIR = './data/MNIST'

In [3]:
def load_data(data_dir, batch_size):
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((160, 160)),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    trainset = torchvision.datasets.MNIST(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.MNIST(
        root=data_dir, train=False, download=True, transform=transform)
    
    train_loader = torch.utils.data.DataLoader(trainset, batch_size)
    test_loader = torch.utils.data.DataLoader(testset, batch_size)

    return train_loader, test_loader

In [4]:
in_channels = 3
out_channels = 10
device = "cuda:0"

train_loss = nn.CrossEntropyLoss()

learning_rate = 1e-4

batch_size = 256
epoch_num = 25

In [5]:
trainset, testset = load_data(data_dir=DATA_DIR, batch_size=batch_size)

In [6]:
def train(net, optimizer, criterion, epoch_num, train_data, device):
    """Neural network training process.
    
    """
    for epoch in range(epoch_num):  # loop over the dataset multiple times
        for data, target in train_data:
            inputs, labels = data.to(device), target.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()    
    return net

In [7]:
class PaddedConv2d(nn.Conv2d):
    def __init__(self, *args, padding=0, **kwargs):
        self.custom_padding = padding
        super().__init__(*args, **kwargs)
        
    def forward(self, x):
        if self.custom_padding:
            x = F.pad(x, (self.custom_padding, self.custom_padding,
                          self.custom_padding, self.custom_padding))
        return super().forward(x)

In [8]:
class Network(nn.Module):
    def __init__(self, channels_list):
        super().__init__()
        channels_list.insert(0, 1)
        self.convs = nn.ModuleList([
            nn.Sequential(
                PaddedConv2d(padding=1, in_channels=in_ch, out_channels=out_ch, kernel_size=3),
                nn.BatchNorm2d(out_ch),
                nn.ReLU()
            )
            for in_ch, out_ch in zip(channels_list[:-1], channels_list[1:])
        ])
        self.out = nn.Linear(in_features=channels_list[-1], out_features=10)
        
    def forward(self, tensor):
        for layer in self.convs:
            tensor = layer(tensor)
            tensor = F.max_pool2d(tensor, kernel_size=2) 
        # output
        tensor = nn.AdaptiveAvgPool2d((1,1))(tensor)
        tensor = torch.flatten(tensor, 1)
        tensor = self.out(tensor)
        tensor = torch.sigmoid(tensor)        
        return tensor

In [9]:
net = Network([8, 16, 32])
net.to(device)

optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [10]:
%%time
net = train(
    net, optimizer, train_loss, epoch_num, trainset, device
)

CPU times: user 4d 10h 6min 2s, sys: 29min 21s, total: 4d 10h 35min 23s
Wall time: 5h 21min 49s


In [11]:
%%time
for data, _ in testset:
    inputs = data.to(device)
    _ = net(inputs)

CPU times: user 42min 35s, sys: 12.3 s, total: 42min 48s
Wall time: 2min 9s
